In [7]:
from sec_edgar_downloader import Downloader

In [8]:
import tqdm
import glob
import re
from bs4 import BeautifulSoup
import os

In [3]:
import pandas as pd

In [10]:
os.listdir()


['functions.py',
 'Main.ipynb',
 '.DS_Store',
 'lowestCosine.csv',
 'Untitled.ipynb',
 'data_transformation.ipynb',
 '__pycache__',
 'README.md',
 'lowestJaccard.csv',
 'SPLatest',
 '.gitignore',
 'matt_notebooks',
 'csv',
 'stocks_formatted.db',
 '.ipynb_checkpoints',
 'sp500_daily.csv',
 '.git']

In [4]:
df = pd.read_csv('csv/sp500.csv')

In [5]:
sp = df.Symbol.to_list()

In [6]:
for i in tqdm.tqdm(sp):
    rml = sp.copy()
    dl=Downloader('SPLatest')
    dl.get_10k_filings(i,2)
    rml.remove(i) #fix half listing

100%|██████████| 505/505 [16:11<00:00,  1.53s/it]


In [26]:
def Clean(soup):
    
    '''
    Removes tables with >15% numerical characters.
    
    Parameters
    ----------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup.
        
    Returns
    -------
    soup : BeautifulSoup object
        Parsed result from BeautifulSoup
        with numerical tables removed.
        
    '''
    
    # Determines percentage of numerical characters
    # in a table
    def GetDigitPercentage(tablestring):
        if len(tablestring)>0.0:
            numbers = sum([char.isdigit() for char in tablestring])
            length = len(tablestring)
            return numbers/length
        else:
            return 1
    
    # Evaluates numerical character % for each table
    # and removes the table if the percentage is > 15%
    [x.extract() for x in soup.find_all('table') if GetDigitPercentage(x.get_text())>0.15]
    
    text = soup.get_text()
    
    # Replace unicode characters with their
    # "normal" representations
    text = unicodedata.normalize('NFKD', text)
    tags = re.compile('<.*?>')
    text = re.sub(tags, ' ', text)
    text = re.sub('[^A-Za-z0-9]+', ' ', text)
    text = text.lower()
    text = " ".join(text.split())
    return text

In [11]:
FilePaths = [str(name) for name in glob.glob('SPLatest/*/*/*/*.txt')]

In [12]:
FilePaths

['SPLatest/sec_edgar_filings/CTAS/10-K/0000723254-17-000020.txt',
 'SPLatest/sec_edgar_filings/CTAS/10-K/0000723254-18-000015.txt',
 'SPLatest/sec_edgar_filings/WELL/10-K/0000766704-18-000012.txt',
 'SPLatest/sec_edgar_filings/WELL/10-K/0000766704-19-000014.txt',
 'SPLatest/sec_edgar_filings/VZ/10-K/0000732712-19-000012.txt',
 'SPLatest/sec_edgar_filings/VZ/10-K/0000732712-18-000009.txt',
 'SPLatest/sec_edgar_filings/AMZN/10-K/0001018724-19-000004.txt',
 'SPLatest/sec_edgar_filings/AMZN/10-K/0001018724-18-000005.txt',
 'SPLatest/sec_edgar_filings/DXC/10-K/0001688568-19-000055.txt',
 'SPLatest/sec_edgar_filings/DXC/10-K/0001688568-18-000036.txt',
 'SPLatest/sec_edgar_filings/CNP/10-K/0001130310-18-000003.txt',
 'SPLatest/sec_edgar_filings/CNP/10-K/0001130310-19-000016.txt',
 'SPLatest/sec_edgar_filings/RCL/10-K/0000884887-19-000017.txt',
 'SPLatest/sec_edgar_filings/RCL/10-K/0000884887-18-000017.txt',
 'SPLatest/sec_edgar_filings/CAT/10-K/0000018230-19-000034.txt',
 'SPLatest/sec_edgar_

In [14]:
def ComputeCosineSimilarity(pair):
# Compile complete set of words in A or B
    text1 = str(BeautifulSoup(open(pair[2]), 'html.parser'))
    text2 = str(BeautifulSoup(open(pair[3]), 'html.parser'))
    words1 = set(text1.split())
    words2 = set(text2.split())
    text1.close()
    text2.close()

    words = set(words1.union(words2))
    
    # Determine which words are in A
    vector_A = [1 if x in words1 else 0 for x in words]
    
    # Determine which words are in B
    vector_B = [1 if x in words2 else 0 for x in words]
    
    # Compute cosine score using scikit-learn
    array_A = np.array(vector_A).reshape(1, -1)
    array_B = np.array(vector_B).reshape(1, -1)
    cosine_score = cosine_similarity(array_A, array_B)[0,0]
    
    return cosine_score


In [19]:
import gensim